In [15]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [17]:
# List of CSV files to merge
csv_files = [
        r'data\2010\temp\temp_data.csv',
        r'data\2011\temp\temp_data.csv',
        r'data\2012\temp\temp_data.csv',
        r'data\2013\temp\temp_data.csv',
        r'data\2014\temp\temp_data.csv',
        r'data\2015\temp\temp_data.csv',
        r'data\2016\temp\temp_data.csv',
        r'data\2017\temp\temp_data.csv',
        r'data\2018\temp\temp_data.csv',
        r'data\2019\temp\temp_data.csv',
        r'data\2020\temp\temp_data.csv',
        r'data\2021\temp\temp_data.csv',
        r'data\2022\temp\temp_data.csv',
        r'data\2023\temp\temp_data.csv',
        r'data\2024\temp\temp_data.csv',
    ]  # Add more files if needed

# Read and merge files
df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV
new_merged_df = pd.concat(df_list, ignore_index=True)

In [18]:
new_merged_df

,valid_time,latitude,longitude,max_temperature,mean_temperature
0,2010-01-01,18.80,72.8,300.55830,298.53610
1,2010-01-01,19.05,72.8,301.07275,298.24356
2,2010-01-02,18.80,72.8,301.43620,299.36893
3,2010-01-02,19.05,72.8,301.72270,299.18018
4,2010-01-03,18.80,72.8,300.03796,298.97437
...,...,...,...,...,...
10953,31-08-24,19.05,72.8,301.93417,300.67007
10954,31-10-24,18.80,72.8,303.40050,302.30900
10955,31-10-24,19.05,72.8,304.09170,302.54822
10956,31-12-24,18.80,72.8,300.60540,299.00990


In [19]:
# # Ensure valid_time is in datetime format
# new_merged_df["valid_time"] = pd.to_datetime(new_merged_df["valid_time"], format="%d-%m-%y", errors="coerce")

# # Extract numeric features from date
# new_merged_df["month"] = new_merged_df["valid_time"].dt.month  # Extract month (1-13)
# new_merged_df["day_of_year"] = new_merged_df["valid_time"].dt.dayofyear 
# new_merged_df["year"] = new_merged_df["valid_time"].dt.year         # Extract Year
#  # Extract day of year (1-365)

# # Drop the original datetime column
# new_merged_df.drop(columns=["valid_time"], inplace=True)

In [20]:
if os.path.exists("data/15year_temperature_without_labelled.csv"):
    print("Files already exist")
else:
    new_merged_df.to_csv("data/15year_temperature_without_labelled.csv")
    print("File created")

Files already exist


In [21]:
df = pd.read_csv("data/15year_temperature_without_labelled.csv")

In [22]:
# df = df.drop(columns=['Unnamed:0'], axis=1)
df

,Unnamed: 0,valid_time,latitude,longitude,max_temperature,mean_temperature
0,0,2010-01-01,18.80,72.8,300.55830,298.53610
1,1,2010-01-01,19.05,72.8,301.07275,298.24356
2,2,2010-01-02,18.80,72.8,301.43620,299.36893
3,3,2010-01-02,19.05,72.8,301.72270,299.18018
4,4,2010-01-03,18.80,72.8,300.03796,298.97437
...,...,...,...,...,...,...
10953,10953,31-08-24,19.05,72.8,301.93417,300.67007
10954,10954,31-10-24,18.80,72.8,303.40050,302.30900
10955,10955,31-10-24,19.05,72.8,304.09170,302.54822
10956,10956,31-12-24,18.80,72.8,300.60540,299.00990


In [23]:
df = df.drop(columns=['Unnamed: 0'], axis=1)

In [25]:
df

,valid_time,latitude,longitude,max_temperature,mean_temperature
0,2010-01-01,18.80,72.8,300.55830,298.53610
1,2010-01-01,19.05,72.8,301.07275,298.24356
2,2010-01-02,18.80,72.8,301.43620,299.36893
3,2010-01-02,19.05,72.8,301.72270,299.18018
4,2010-01-03,18.80,72.8,300.03796,298.97437
...,...,...,...,...,...
10953,31-08-24,19.05,72.8,301.93417,300.67007
10954,31-10-24,18.80,72.8,303.40050,302.30900
10955,31-10-24,19.05,72.8,304.09170,302.54822
10956,31-12-24,18.80,72.8,300.60540,299.00990


In [24]:
X = df.drop(columns=['heatwave_label'], axis=1)
X

KeyError: "['heatwave_label'] not found in axis"

In [ ]:
y = df['heatwave_label']
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

In [ ]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "LDA":LinearDiscriminantAnalysis(),
    "Lightgbm": LGBMClassifier(),
    "Ridge": RidgeClassifier(),
    "MLP": MLPClassifier(),
    "MultimodalNB": MultinomialNB(),
    "GaussianNB": GaussianNB(),
    "Bernoulli": BernoulliNB(),
    "K-Neighbors Classifier": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest Classifier": RandomForestClassifier(),
    "XGBClassifier": XGBClassifier(), 
    "CatBoosting Classifier": CatBoostClassifier(verbose=False),
    "AdaBoost Classifier": AdaBoostClassifier(),
    "SVM": SVC()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('\n')


In [ ]:

pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
